# Availability-Based Methods  
In this notebook, I attempt to implement a method that checks whether an outfit is available for rent.  

This approach is considered unsuccessful, as I was unable to find a way to evaluate its effect.

In [ ]:
#import
import pandas as pd
import numpy as np
from datetime import datetime

"""use_testing_time is a parameter that determines whether we are currently testing the system or making real predictions
    (true is used to test the system)"""

def availability_based_filter(df1, df2=None, use_testing_time=True):

    # Function to remove ordered items from lists during testing time
    def remove_ordered_items(row, orders_df=None, use_testing_time=True):
        items_list = row['id_prediction']
        testing_times = row['test_booking_time_start']

        if use_testing_time:
            order_start_times = row['train_booking_times_start']
            order_end_times = row['train_booking_times_end']

            # Filter orders that happened before or at the testing times
            orders_during_testing = [i for i in range(len(order_start_times)) if any(order_start_times[i] <= testing_time <= order_end_times[i] for testing_time in testing_times)]

            # Remove ordered items from the list
            updated_list = [item for i, item in enumerate(items_list) if i not in orders_during_testing]
        else:
            current_time = datetime.now()

            # Filter orders that happened before or at the current time
            orders_during_testing = orders_df[(orders_df['rentalPeriod.start'] <= current_time) & (orders_df['rentalPeriod.end'] >= current_time)]

            # Remove ordered items from the list
            updated_list = [item for item in items_list if item not in orders_during_testing['outfit.id	'].values]

        return updated_list

    # Apply the function to each row in df1
    df1['updated_lists'] = df1.apply(remove_ordered_items, axis=1, orders_df=df2, use_testing_time=use_testing_time)

    return df1
